# Coursera: Applied Data Science Capstone

In this notebook I will document all steps to completing the Coursera Applied Data Science Capstone.

In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [2]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


# Segmenting and Clustering Neighborhoods in Toronto

## Part 1: Get postal codes and neighbourhoods

In [3]:
website_url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

In [4]:
# read table from Wikipedia
postal_codes = pd.read_html(website_url)[0]

In [5]:
# drop rows that do not have a borough
postal_codes = postal_codes[postal_codes.Borough != "Not assigned"].reset_index(drop = True)

In [6]:
# combine neighbourhoods within the same borough and postcode
df_postal = pd.DataFrame(postal_codes.groupby(["Postcode","Borough"],as_index=False)["Neighbourhood"].apply(list)).reset_index()
df_postal.rename(columns={0:"Neighbourhood"}, inplace = True)
df_postal.Neighbourhood = df_postal.Neighbourhood.apply(lambda x: ', '.join([str(i) for i in x]))

In [7]:
# check which entries do not have a neighborhood
df_postal[df_postal.Neighbourhood == "Not assigned"]

# only Queen's Park is affected so we can simply replace it
df_postal = df_postal.replace(to_replace="Not assigned", value="Queen's Park")

### Results 

In [8]:
df_postal

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv..."
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ..."


In [9]:
df_postal.shape

(103, 3)

## Part 2: Get geo coordinates

In [15]:
coords = pd.read_csv("Geospatial_Coordinates.csv")
coords

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [21]:
df_coords = df_postal.merge(coords, how = "left", left_on = "Postcode", right_on = "Postal Code")
df_coords.drop("Postal Code", axis = 1, inplace = True)
df_coords

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv...",43.688905,-79.554724
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ...",43.739416,-79.588437


In [4]:
# Save data so previous steps need not be repeated each time
df_coords.to_csv("df_coords.csv")

NameError: name 'df_coords' is not defined

## Part 3: Clustering

In [2]:
df_coords = pd.read_csv("df_coords.csv", index_col="Unnamed: 0")
df_coords.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


Find location of Toronto:

In [3]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
toronto_lat = location.latitude
toronto_lon = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(toronto_lat, toronto_lon))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


### Create a map of Toronto and its neighbourhoods

In [4]:
map_toronto = folium.Map(location=[toronto_lat, toronto_lon], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_coords['Latitude'], df_coords['Longitude'], df_coords['Borough'], df_coords['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Define Foursquare credentials:

In [ ]:
CLIENT_ID = 'test' # your Foursquare ID
CLIENT_SECRET = 'test' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

### Get the top 100 venues in each neighbourhood

In [6]:
limit = 100
radius = 500

In [7]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [10]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
toronto_venues = getNearbyVenues(names=df_coords['Neighbourhood'],
                                   latitudes=df_coords['Latitude'],
                                   longitudes=df_coords['Longitude']
                                  )

In [14]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge, Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
3,"Guildwood, Morningside, West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Marina Spa,43.766000,-79.191000,Spa


In [12]:
print(toronto_venues.shape)

(2204, 7)


### Let's limit the dataset to those neighborhoods with at least 10 venues.

In [22]:
neighborhood_count = toronto_venues.groupby('Neighborhood').count()[["Venue"]]
neighborhoods = neighborhood_count[neighborhood_count.Venue >= 10].index.values

In [25]:
toronto_venues = toronto_venues[toronto_venues.Neighborhood.isin(neighborhoods)]
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
60,"Clarks Corners, Sullivan, Tam O'Shanter",43.781638,-79.304302,Remezzo Italian Bistro,43.778649,-79.308264,Italian Restaurant
61,"Clarks Corners, Sullivan, Tam O'Shanter",43.781638,-79.304302,The Royal Chinese Restaurant 避風塘小炒,43.780505,-79.298844,Chinese Restaurant
62,"Clarks Corners, Sullivan, Tam O'Shanter",43.781638,-79.304302,Eight Noodles,43.778234,-79.308299,Noodle House
63,"Clarks Corners, Sullivan, Tam O'Shanter",43.781638,-79.304302,Kub Khao,43.780438,-79.299837,Thai Restaurant
64,"Clarks Corners, Sullivan, Tam O'Shanter",43.781638,-79.304302,TD Canada Trust,43.779169,-79.303617,Bank


In [26]:
print(toronto_venues.shape)

(1990, 7)


### Analyze neighborhoods

In [27]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Social Club,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
60,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Clarks Corners, Sullivan, Tam O'Shanter",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
61,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [28]:
toronto_onehot.shape

(1990, 240)

In [30]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Social Club,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,"Adelaide, King, Richmond",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.02,0.0,0.0,0.000000,0.0,0.03,0.0,0.0,0.000000,0.0,0.000000,0.030000,0.000000,0.04,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.01,0.0,0.01,0.010000,0.0,0.000000,0.0,0.01,0.03,0.01,0.0,0.0,0.0,0.000000,0.040000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.01,0.000000,0.070000,0.0,0.0,0.0,0.0,0.01,0.000000,0.0,0.020000,0.0,0.03,0.0,0.000000,0.0,0.0,0.0,0.0,0.010000,0.010000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.01,0.0,0.0,0.0,0.000000,0.00,0.0,0.0,0.000000,0.0,0.0,0.0,0.01

In [31]:
toronto_grouped.shape

(48, 240)

In [32]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [33]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Steakhouse,Thai Restaurant,Bar,Café,Asian Restaurant,Bakery,Cosmetics Shop,Restaurant,Burger Joint
1,"Albion Gardens, Beaumond Heights, Humbergate, ...",Grocery Store,Coffee Shop,Fast Food Restaurant,Liquor Store,Beer Store,Pharmacy,Pizza Place,Fried Chicken Joint,Sandwich Place,Dance Studio
2,"Bathurst Manor, Downsview North, Wilson Heights",Coffee Shop,Middle Eastern Restaurant,Diner,Sandwich Place,Deli / Bodega,Bridal Shop,Restaurant,Bank,Ice Cream Shop,Supermarket
3,"Bedford Park, Lawrence Manor East",Thai Restaurant,Sandwich Place,Coffee Shop,Italian Restaurant,Fast Food Restaurant,Greek Restaurant,Juice Bar,Liquor Store,Restaurant,Indian Restaurant
4,Berczy Park,Coffee Shop,Seafood Restaurant,Bakery,Café,Steakhouse,Beer Bar,Cheese Shop,Cocktail Bar,Farmers Market,Italian Restaurant


### Clustering

In [34]:
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

kmeans.labels_[0:10] 

array([0, 4, 1, 1, 0, 0, 2, 0, 0, 1], dtype=int32)

In [40]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df_coords

toronto_merged = toronto_merged.merge(neighborhoods_venues_sorted.set_index('Neighborhood'), left_on='Neighbourhood', right_on="Neighborhood")

toronto_merged.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1T,Scarborough,"Clarks Corners, Sullivan, Tam O'Shanter",43.781638,-79.304302,2,Pizza Place,Fried Chicken Joint,Bank,Intersection,Gas Station,Noodle House,Chinese Restaurant,Fast Food Restaurant,Italian Restaurant,Thai Restaurant
1,M1W,Scarborough,L'Amoreaux West,43.799525,-79.318389,4,Chinese Restaurant,Fast Food Restaurant,Coffee Shop,Grocery Store,Pharmacy,Pizza Place,Sandwich Place,Breakfast Spot,Nail Salon,Women's Store
2,M2J,North York,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,3,Clothing Store,Coffee Shop,Fast Food Restaurant,Japanese Restaurant,Women's Store,Bakery,Juice Bar,Food Court,Tea Room,Convenience Store
3,M2N,North York,Willowdale South,43.770120,-79.408493,2,Ramen Restaurant,Café,Sandwich Place,Japanese Restaurant,Restaurant,Pizza Place,Coffee Shop,Sushi Restaurant,Electronics Store,Fast Food Restaurant
4,M3C,North York,"Flemingdon Park, Don Mills South",43.725900,-79.340923,3,Beer Store,Coffee Shop,Clothing Store,Gym,Asian Restaurant,Japanese Restaurant,Supermarket,Bus Line,Fast Food Restaurant,Sporting Goods Shop


### Map clusters

In [41]:
# create map
map_clusters = folium.Map(location=[toronto_lat, toronto_lon], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine clusters

In [43]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,"The Danforth West, Riverdale",0,Greek Restaurant,Italian Restaurant,Coffee Shop,Bookstore,Ice Cream Shop,Furniture / Home Store,Fruit & Vegetable Store,Pub,Pizza Place,Liquor Store
12,Studio District,0,Café,Coffee Shop,Bakery,Italian Restaurant,Brewery,American Restaurant,Gastropub,Yoga Studio,Clothing Store,Bookstore
16,"Cabbagetown, St. James Town",0,Coffee Shop,Bakery,Café,Chinese Restaurant,Restaurant,Italian Restaurant,Pizza Place,Park,Pub,Market
17,Church and Wellesley,0,Coffee Shop,Gay Bar,Japanese Restaurant,Sushi Restaurant,Restaurant,Gym,Pub,Men's Store,Mediterranean Restaurant,Hotel
18,Harbourfront,0,Coffee Shop,Bakery,Pub,Café,Park,Theater,Mexican Restaurant,Breakfast Spot,Shoe Store,Restaurant
20,St. James Town,0,Café,Coffee Shop,Restaurant,Beer Bar,Clothing Store,Cocktail Bar,Bakery,Breakfast Spot,Cosmetics Shop,Italian Restaurant
21,Berczy Park,0,Coffee Shop,Seafood Restaurant,Bakery,Café,Steakhouse,Beer Bar,Cheese Shop,Cocktail Bar,Farmers Market,Italian Restaurant
23,"Adelaide, King, Richmond",0,Coffee Shop,Steakhouse,Thai Restaurant,Bar,Café,Asian Restaurant,Bakery,Cosmetics Shop,Restaurant,Burger Joint
24,"Harbourfront East, Toronto Islands, Union Station",0,Coffee Shop,Aquarium,Hotel,Café,Italian Restaurant,Fried Chicken Joint,Scenic Lookout,Brewery,Restaurant,Bar
25,"Design Exchange, Toronto Dominion Centre",0,Coffee Shop,Café,Restaurant,Hotel,Italian Restaurant,American Restaurant,Bar,Gastropub,Steakhouse,Deli / Bodega


In [44]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,"Bathurst Manor, Downsview North, Wilson Heights",1,Coffee Shop,Middle Eastern Restaurant,Diner,Sandwich Place,Deli / Bodega,Bridal Shop,Restaurant,Bank,Ice Cream Shop,Supermarket
9,Thorncliffe Park,1,Indian Restaurant,Yoga Studio,Supermarket,Gas Station,Grocery Store,Gym,Fast Food Restaurant,Liquor Store,Middle Eastern Restaurant,Coffee Shop
14,Davisville,1,Dessert Shop,Sandwich Place,Sushi Restaurant,Italian Restaurant,Café,Pizza Place,Gym,Coffee Shop,Gourmet Shop,Park
15,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",1,Pub,Coffee Shop,Sports Bar,Sushi Restaurant,Vietnamese Restaurant,Light Rail Station,Liquor Store,Pizza Place,Supermarket,American Restaurant
22,Central Bay Street,1,Coffee Shop,Italian Restaurant,Burger Joint,Café,Juice Bar,Japanese Restaurant,Ice Cream Shop,Sandwich Place,Bar,Bubble Tea Shop
27,"Bedford Park, Lawrence Manor East",1,Thai Restaurant,Sandwich Place,Coffee Shop,Italian Restaurant,Fast Food Restaurant,Greek Restaurant,Juice Bar,Liquor Store,Restaurant,Indian Restaurant
28,"The Annex, North Midtown, Yorkville",1,Coffee Shop,Sandwich Place,Café,Park,Cosmetics Shop,Liquor Store,Burger Joint,Indian Restaurant,Middle Eastern Restaurant,BBQ Joint
42,Queen's Park,1,Coffee Shop,Gym,Park,Yoga Studio,College Auditorium,Seafood Restaurant,Sandwich Place,Salad Place,Burger Joint,Burrito Place
43,Queen's Park,1,Coffee Shop,Gym,Park,Yoga Studio,College Auditorium,Seafood Restaurant,Sandwich Place,Salad Place,Burger Joint,Burrito Place
44,Canada Post Gateway Processing Centre,1,Coffee Shop,Hotel,American Restaurant,Mediterranean Restaurant,Burrito Place,Intersection,Fried Chicken Joint,Sandwich Place,Middle Eastern Restaurant,Gym


In [46]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Clarks Corners, Sullivan, Tam O'Shanter",2,Pizza Place,Fried Chicken Joint,Bank,Intersection,Gas Station,Noodle House,Chinese Restaurant,Fast Food Restaurant,Italian Restaurant,Thai Restaurant
3,Willowdale South,2,Ramen Restaurant,Café,Sandwich Place,Japanese Restaurant,Restaurant,Pizza Place,Coffee Shop,Sushi Restaurant,Electronics Store,Fast Food Restaurant
7,Woodbine Heights,2,Beer Store,Pharmacy,Spa,Curling Ice,Skating Rink,Bus Stop,Cosmetics Shop,Athletics & Sports,Park,Video Store
11,"The Beaches West, India Bazaar",2,Park,Pub,Liquor Store,Sandwich Place,Brewery,Italian Restaurant,Burger Joint,Burrito Place,Ice Cream Shop,Steakhouse
45,Business Reply Mail Processing Centre 969 Eastern,2,Light Rail Station,Yoga Studio,Pizza Place,Smoke Shop,Skate Park,Brewery,Restaurant,Recording Studio,Burrito Place,Comic Shop
46,"Humber Bay Shores, Mimico South, New Toronto",2,Café,Pizza Place,Restaurant,Fast Food Restaurant,Hobby Shop,Fried Chicken Joint,Bakery,Mexican Restaurant,Gym,Pharmacy
47,"Kingsway Park South West, Mimico NW, The Queen...",2,Bakery,Supplement Shop,Discount Store,Wings Joint,Grocery Store,Gym,Sandwich Place,Burrito Place,Hardware Store,Social Club


In [47]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,"Fairview, Henry Farm, Oriole",3,Clothing Store,Coffee Shop,Fast Food Restaurant,Japanese Restaurant,Women's Store,Bakery,Juice Bar,Food Court,Tea Room,Convenience Store
4,"Flemingdon Park, Don Mills South",3,Beer Store,Coffee Shop,Clothing Store,Gym,Asian Restaurant,Japanese Restaurant,Supermarket,Bus Line,Fast Food Restaurant,Sporting Goods Shop
8,Leaside,3,Coffee Shop,Sporting Goods Shop,Sushi Restaurant,Furniture / Home Store,Burger Joint,Electronics Store,Bagel Shop,Bank,Sports Bar,Liquor Store
13,North Toronto West,3,Clothing Store,Coffee Shop,Fast Food Restaurant,Park,Chinese Restaurant,Café,Mexican Restaurant,Rental Car Location,Restaurant,Salon / Barbershop
19,"Ryerson, Garden District",3,Coffee Shop,Clothing Store,Japanese Restaurant,Café,Cosmetics Shop,Electronics Store,Lingerie Store,Italian Restaurant,Middle Eastern Restaurant,Ice Cream Shop
34,"Lawrence Heights, Lawrence Manor",3,Clothing Store,Furniture / Home Store,Gift Shop,Miscellaneous Shop,Boutique,Event Space,Sporting Goods Shop,Arts & Crafts Store,Coffee Shop,Vietnamese Restaurant


In [48]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,L'Amoreaux West,4,Chinese Restaurant,Fast Food Restaurant,Coffee Shop,Grocery Store,Pharmacy,Pizza Place,Sandwich Place,Breakfast Spot,Nail Salon,Women's Store
6,"Woodbine Gardens, Parkview Hill",4,Fast Food Restaurant,Pizza Place,Bank,Bus Line,Breakfast Spot,Gastropub,Pet Store,Gym / Fitness Center,Pharmacy,Athletics & Sports
48,"Albion Gardens, Beaumond Heights, Humbergate, ...",4,Grocery Store,Coffee Shop,Fast Food Restaurant,Liquor Store,Beer Store,Pharmacy,Pizza Place,Fried Chicken Joint,Sandwich Place,Dance Studio
